In [1]:
#Example of how to get Landsat data using the getImagesLib, create median composites, run LandTrendr and then filter
#LandTrendr output into usable data depicting where, when, and the magnitude of loss and gain
####################################################################################################

try:
  import geeViz.getImagesLib as getImagesLib
except:
  !python -m pip install geeViz
  import geeViz.getImagesLib as getImagesLib


#Module imports
# import geeViz.getImagesLib as getImagesLib
import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
ee = getImagesLib.ee

#Set up to mapper object
Map = getImagesLib.Map


print('done')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.4/676.4 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.5 MB/s eta 0:00:00
  Created wheel for simpledbf: filename=simpledbf-0.2.6-py3-none-any.whl size=13785 sha256=63b03446b951055cfd822e39f0bea9a097064c6ffec8694218fff92b176cdb97
  Stored in directory: /root/.cache/pip/wheels/e5/41/13/ebdef29165b9309ec4e235dbff19eca8b6759125b0924ad430
Successfully built simpledbf
Initializing GEE
No credentials found
Will attempt ee.Authenticate
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=B4C6K1JE1gCyc6UJNbM9WIz5-kcJWciaoq91-gxYKhk&tc=Ycg-X4Zer1bCwiip

In [2]:
Map.clearMap()
# Pick up from module 1 by bringing in composite asset
compositesPath = 'projects/rcr-gee/assets/composites-lcms-training-module-1'

# Index to use to demonstrate change detection capabilities of LandTrendr
#Choose band or index
#NBR, NDMI, and NDVI tend to work best
#Other good options are wetness and tcAngleBG
# Must include 'red','nir','swir1' in order to visualize as false color composites in next code block
bandNames = ['red','nir','swir1','swir2','NBR','NDVI']

#Define landtrendr params
run_params = { \
  'maxSegments':            6,\
  'spikeThreshold':         0.9,\
  'vertexCountOvershoot':   3,\
  'preventOneYearRecovery': False,\
  'recoveryThreshold':      0.25,\
  'pvalThreshold':          0.05,\
  'bestModelProportion':    0.75,\
  'minObservationsNeeded':  6\
}


#Set up Names for the export
outputName = 'LT_Test'

#Provide location composites will be exported to
#This should be an asset folder, or more ideally, an asset imageCollection
exportPathRoot = 'projects/rcr-gee/assets/landTrendr-lcms-training-module-2'
# Make sure collection exists
aml.create_image_collection(exportPathRoot)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(exportPathRoot,writers = [],all_users_can_read = True,readers = [])

# Bring in composites and pull info from them
composites = ee.ImageCollection(compositesPath)

props = composites.first().toDictionary().getInfo()

startYear = props['startYear']
endYear = props['endYear']

startJulian = props['startJulian']
endJulian = props['endJulian']


proj = composites.first().projection().getInfo()
crs = proj['crs']
transform = proj['transform']
scale = None

studyArea = composites.first().geometry()

# Decompress composites by dividing by 10000 for optical bands and add indices
composites = composites.select(['blue','green','red','nir','swir1','swir2']).map(lambda img: img.divide(10000).float().copyProperties(img,['system:time_start']))
composites = composites.map(getImagesLib.simpleAddIndices)\
                      .map(getImagesLib.getTasseledCap)\
                      .map(getImagesLib.simpleAddTCAngles)

print('done')

Collection projects/rcr-gee/assets/landTrendr-lcms-training-module-2 already exists
Updating permissions for:  projects/rcr-gee/assets/landTrendr-lcms-training-module-2
done


In [3]:
# First let's explore the composites and some attributes that can help understand how well the composites turned out
Map.clearMap()
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse,'Composites')

Map.turnOnInspector()
Map.view()

Adding layer: Composites
Starting webmap
Using default refresh token for geeView: /root/.config/earthengine/credentials
Starting local web server at: http://localhost:8001/geeView/
HTTP server command: "/usr/bin/python3" -m http.server  8001
Done
cwd /content
Colab Proxy URL: https://1a674su2mzc-496ff2e9c6d22116-8001-colab.googleusercontent.com/geeView/?accessToken=None


In [4]:
# We will run LandTrendr for each band
#Clear the map in case it has been populated with layers/commands earlier
Map.clearMap()

#Run LANDTRENDR
for bandName in bandNames:

  # Select the band and run LandTrendr
  run_params['timeSeries'] = composites.select([bandName])
  rawLT = ee.Algorithms.TemporalSegmentation.LandTrendr(**run_params)

  Map.addLayer(rawLT,{},'LT Raw {}'.format(bandName),False)

  # Notice the raw LandTrendr output is in GEE's image array format
  # We'll need to manipulate the raw output a bit to save on storage space
  # Mask out non vertex values to use less storage space
  ltArray = rawLT.select(['LandTrendr'])
  rmse = rawLT.select(['rmse'])
  vertices = ltArray.arraySlice(0,3,4)
  ltArray = ltArray.arrayMask(vertices)

  # Mask out all but the year and vertex fited values (get rid of the raw and vertex rows)
  ltArray = ltArray.arrayMask(ee.Image(ee.Array([[1],[0],[1],[0]])))
  rawLTForExport=ltArray.addBands(rmse)
  Map.addLayer(rawLTForExport,{},'LT Vertex Values Only {}'.format(bandName),False)

  # Show how the compressed vertex-only values can be decompressed later
  decompressedC = changeDetectionLib.simpleLTFit(ltArray,startYear,endYear,bandName,True,run_params['maxSegments'])
  Map.addLayer(decompressedC,{'bands':'{}_LT_fitted'.format(bandName),'min':0.2,'max':0.8},'Decompressed LT Output {}'.format(bandName),False)

  # Join the raw and fited values
  fitted = decompressedC.select(['{}_LT_fitted'.format(bandName)])
  ltJoined = getImagesLib.joinCollections(composites.select([bandName]),fitted)
  Map.addLayer(ltJoined,{'bands':'{}_LT_fitted'.format(bandName),'min':0.2,'max':1,'palette':'D80,080'},'Raw and LT Fitted {}'.format(bandName),True)

  # Export LT array image
  # Set some properties that will be uses later
  rawLTForExport = rawLTForExport.set({'startYear':startYear,
                                          'endYear':endYear,
                                          'startJulian':startJulian,
                                          'endJulian':endJulian,
                                          'band':bandName})
  rawLTForExport =rawLTForExport.set(run_params)
  exportName = 'LT_Raw_{}_yrs{}-{}_jds{}-{}'.format(bandName,startYear,endYear,startJulian,endJulian)
  exportPath = exportPathRoot + '/'+ exportName
  # Export output
  getImagesLib.exportToAssetWrapper(rawLTForExport,exportName,exportPath,{'.default':'sample'},studyArea,scale,crs,transform,overwrite=True)

Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

Adding layer: LT Raw red
Adding layer: LT Vertex Values Only red
Adding layer: Decompressed LT Output red
Adding layer: Raw and LT Fitted red
pyramiding object: {'.default': 'sample'}
Exporting: LT_Raw_red_yrs2010-2022_jds273-272
<Task EXPORT_IMAGE: LT_Raw_red_yrs2010-2022_jds273-272 (UNSUBMITTED)>
Adding layer: LT Raw nir
Adding layer: LT Vertex Values Only nir
Adding layer: Decompressed LT Output nir
Adding layer: Raw and LT Fitted nir
pyramiding object: {'.default': 'sample'}
Exporting: LT_Raw_nir_yrs2010-2022_jds273-272
<Task EXPORT_IMAGE: LT_Raw_nir_yrs2010-2022_jds273-272 (UNSUBMITTED)>
Adding layer: LT Raw swir1
Adding layer: LT Vertex Values Only swir1
Adding layer: Decompressed LT Output swir1
Adding layer: Raw and LT Fitted swir1
pyramiding object: {'.default': 'sample'}
Exporting: LT_Raw_swir1_yrs2010-2022_jds273-272
<Task EXPORT_IMAGE: LT_Raw_swir1_yrs2010-2022_jds273-272 (UNSUBMITTED)>
Adding layer: LT Raw swir2
Adding layer: LT Vertex Values Only swir2
Adding layer: Decom

In [25]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
tml.trackTasks2()

0 tasks ready 2023-07-24 15:10:57
0 tasks running 2023-07-24 15:10:57
Running names:




In [5]:
Map.clearMap()

# View exported LT output
lt = ee.ImageCollection(exportPathRoot)
# Convert stacked outputs into collection of fitted, magnitude, slope, duration, etc values for each year
lt_fit = changeDetectionLib.batchSimpleLTFit(lt,startYear,endYear,None,bandPropertyName='band',arrayMode=True)

# Vizualize image collection for charting (opacity set to 0 so it will chart but not be visible)
Map.addLayer(lt_fit,{'opacity':0},'LT Fit TS')

# Visualize fitted landTrendr composites
fitted_bns = lt_fit.select(['.*_fitted']).first().bandNames()
out_bns = fitted_bns.map(lambda bn: ee.String(bn).split('_').get(0))

# Give same names as composites
lt_synth = lt_fit.select(fitted_bns,out_bns)

# Visualize raw and LandTrendr fitted composites
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse,'Raw Composite Timelapse')

Map.addTimeLapse(lt_synth,getImagesLib.vizParamsFalse,'Synthetic Composite Timelapse')


# Join the raw and fited values
ltJoined = getImagesLib.joinCollections(composites.select(bandNames),lt_fit.select(['.*_fitted']))
print(ltJoined.first().bandNames().getInfo())
Map.addLayer(ltJoined,{'min':0.2,'max':1},'Raw and LT Fitted',True)


Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

Adding layer: LT Fit TS
Adding layer: Raw Composite Timelapse
Adding layer: Synthetic Composite Timelapse
['red', 'nir', 'swir1', 'swir2', 'NBR', 'NDVI', 'NBR_LT_fitted', 'NDVI_LT_fitted', 'nir_LT_fitted', 'red_LT_fitted', 'swir1_LT_fitted', 'swir2_LT_fitted']
Adding layer: Raw and LT Fitted
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: /root/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /content
Colab Proxy URL: https://1a674su2mzc-496ff2e9c6d22116-8001-colab.googleusercontent.com/geeView/?accessToken=None
